In [1]:
import json
from pathlib import Path
from pprint import pprint
from io import BytesIO

import h5py
from PIL import Image
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model
from accelerate import Accelerator

from isic_helper import get_folds

In [2]:
model_name = "efficientnet_b4"
version = "v1"
path = f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train"

In [3]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"

In [4]:
def test_augment(image_size):
    transform = A.Compose(
        [A.Resize(image_size, image_size), A.Normalize(), ToTensorV2()], p=1.0
    )
    return transform


class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        row = self.metadata.iloc[index]
        image = np.array(Image.open(BytesIO(self.images[row["isic_id"]][()])))
        if self.augment is not None:
            image = self.augment(image=image)["image"].float()
        if self.infer:
            return image
        else:
            target = torch.tensor(row["target"]).float()
            return image, target



class ISICNet(nn.Module):
    def __init__(
        self,
        model_name,
        pretrained=True,
    ):
        super(ISICNet, self).__init__()
        self.model = create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=3,
            num_classes=0,
            global_pool="",
        )
        in_dim = self.model.num_features
        self.classifier = nn.Linear(in_dim, 1)
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])

    def forward(self, images):
        x = self.model(images)
        bs = len(images)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        if self.training:
            logits = 0
            for i in range(len(self.dropouts)):
                logits += self.classifier(self.dropouts[i](pool))
            logits = logits / len(self.dropouts)
        else:
            logits = self.classifier(pool)
        return logits


def get_trans(img, iteration):
    if iteration >= 6:
        img = img.transpose(2, 3)
    if iteration % 6 == 0:
        return img
    elif iteration % 6 == 1:
        return torch.flip(img, dims=[2])
    elif iteration % 6 == 2:
        return torch.flip(img, dims=[3])
    elif iteration % 6 == 3:
        return torch.rot90(img, 1, dims=[2, 3])
    elif iteration % 6 == 4:
        return torch.rot90(img, 2, dims=[2, 3])
    elif iteration % 6 == 5:
        return torch.rot90(img, 3, dims=[2, 3])


def predict(model, test_dataloader, accelerator, n_tta, log_interval=10):
    model.eval()
    test_probs = []
    total_steps = len(test_dataloader)
    with torch.no_grad():
        for step, images in enumerate(test_dataloader):
            logits = 0
            probs = 0
            for i in range(n_tta):
                logits_iter = model(get_trans(images, i))
                logits += logits_iter
                probs += torch.sigmoid(logits_iter)
            logits /= n_tta
            probs /= n_tta

            probs = accelerator.gather(probs)
            test_probs.append(probs)

            if (step == 0) or ((step + 1) % log_interval == 0):
                print(
                    f"Step: {step + 1}/{total_steps}"
                )

    test_probs = torch.cat(test_probs).cpu().numpy()
    return test_probs

In [5]:
INPUT_PATH = Path("../input/isic-2024-challenge/")

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

Train data size: (401059, 57)
Test data size: (3, 44)


In [6]:
def get_dnn_predictions(train, test, test_images, model_name, version, path):
    with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
        run_metadata = json.load(f)
    pprint(run_metadata["params"])
    
    test_dataset = ISICDataset(
        test, test_images, augment=test_augment(run_metadata["params"]["image_size"]), infer=True
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=512,
        shuffle=False,
        num_workers=4,
        drop_last=False,
        pin_memory=True,
    )
    
#     all_folds = np.unique(train["fold"])
    all_folds = [1]
    test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
    for fold in all_folds:
        accelerator = Accelerator(
            mixed_precision=run_metadata["params"]["mixed_precision"],
        )
        
        model = ISICNet(model_name=model_name, pretrained=False)
        model = model.to(accelerator.device)
        
        model, test_dataloader = accelerator.prepare(model, test_dataloader)
        model_filepath = path / f"models/fold_{fold}"
        accelerator.load_state(model_filepath)

        test_predictions_df[f"fold_{fold}"] = predict(model, test_dataloader, accelerator, n_tta=run_metadata["params"]["n_tta"])
    test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
    return test_predictions_df[[id_column, target_column]]

In [7]:
test_preds_df = get_dnn_predictions(train_metadata, test_metadata, test_images, model_name, version, Path(path))

{'debug': True,
 'ext': '2020,2019',
 'image_size': 64,
 'init_lr': 3e-05,
 'mixed_precision': 'fp16',
 'n_tta': 8,
 'num_epochs': 20,
 'num_workers': 8,
 'only_malignant': True,
 'seed': 2022,
 'train_batch_size': 64,
 'val_batch_size': 512}
Step: 1/1


In [8]:
test_preds_df.head()

,isic_id,target
0,ISIC_0015657,0.509826
1,ISIC_0015729,0.470818
2,ISIC_0015740,0.563106


In [9]:
test_preds_df[target_column].describe()

count    3.000000
mean     0.514583
std      0.046328
min      0.470818
25%      0.490322
50%      0.509826
75%      0.536466
max      0.563106
Name: target, dtype: float64

In [10]:
test_preds_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_0015657,0.509826
1,ISIC_0015729,0.470818
2,ISIC_0015740,0.563106


In [11]:
test_preds_df[[id_column, target_column]].to_csv("submission.csv", index=False)